Zhang Chao, 2025.04.25<br>
Three components of ET changes

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import regionmask
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import sys
import scipy
import seaborn as sns
from matplotlib.colors import Normalize
import cartopy.io.shapereader as shpreader
import matplotlib.colors as mcolors
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from utils import getSigFlag, stats_regionmean
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')

dLE = xr.open_dataset('delta_LE_Yr_CN_2001_2020_PMLv2.nc')
dEVI = xr.open_dataset('delta_EVI_Yr_CN_2001_2020.nc')

shp_cn = gpd.read_file('../shapefile/ChinaAll.shp')
shp_nanhai = gpd.read_file('../shapefile/Nanhai.shp')
shp_climzone = gpd.read_file('../shapefile/ClimateZone_3.shp')

In [3]:
df_dLE = stats_regionmean(dLE,'LE',shp_climzone)
df_dEVI = stats_regionmean(dEVI,'EVI',shp_climzone)

In [4]:
def plot_curves(ax,y,color,xticklabels,title,no):
    x = range(0,y.shape[0])
    ax.plot(x,y,color=color)
    parameter = np.polyfit(x, y, 1)
    f = np.poly1d(parameter)
    r, sig = scipy.stats.pearsonr(x, y)
    ax.plot(x, f(x), ls='--', c=color)
    ax.text(0.10,0.90, "slope = %.2f$^{%s}$" %
             (parameter[0]*10, getSigFlag(sig)),
             transform=ax.transAxes, c= 'k',fontsize=12)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xticks([4,14],xticklabels)

    ax.set_title(title)
    ax.text(-0.1,1.01,no,transform=ax.transAxes,fontsize=14,weight='bold')
    
    
def drawScatterPlots(fig,axpos,x,y,no):
    ax=fig.add_axes(axpos)
    sns.regplot(x=x, y=y,ax=ax,marker='o',color='k',label='Daytime',
                line_kws = {'linewidth':2},scatter_kws={'s':12})
    parameter = np.polyfit(x,y,1)
    f = np.poly1d(parameter)
    r, sig = scipy.stats.pearsonr(x, y)
    ax.text(0.08, 0.90, "R\u00b2 = ",transform=ax.transAxes,fontsize=12)
    # ax.text(0.55, 0.90, ",", transform=ax.transAxes,fontsize=12)
    ax.text(0.28, 0.90, "%.2f$^{%s}$" % (r * r, getSigFlag(sig)),
             transform=ax.transAxes, c='k',fontsize=12)

    ax.text(-0.1,1.01,no,transform=ax.transAxes,fontsize=14,weight='bold')
    
    ax.set_ylabel('')
    ax.set_xlabel('$\Delta$EVI')
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=4,figsize=(10,6.5),sharex=False)
plot_settings()
x = range(0,df_dLE.shape[0])
colors = ['g','#1a8076','k']
titles =['China','Arid','Semi-arid/humid','Humid']
titles2=['','','','']

Nos = ['a','b','c','d','e','f','g','h','i','j','k','l']
for i in range(4):
    plot_curves(axes[0,i],df_dEVI.iloc[:,i+1]*1000,colors[0],['2005','2015'],titles[i],Nos[i])
    plot_curves(axes[1,i],df_dLE.iloc[:,i+1],colors[1],['2005','2015'],titles2[i],Nos[4+i])
    drawScatterPlots(fig,axes[2,i],df_dEVI.iloc[:,i+1]*1000,df_dLE.iloc[:,i+1],Nos[8+i])

axes[0,0].set_ylabel('$\Delta$EVI')
axes[1,0].set_ylabel('$\Delta$LE [W/m$^{2}$]')
axes[2,0].set_ylabel('$\Delta$LE [W/m$^{2}$]')


plt.tight_layout()
plt.savefig('../figures/Figure_S05.png',dpi=300)